In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

example_sent = "This is a sample sentence, showing off the stop words filtration."

stop_words = set(stopwords.words('english'))

word_tokens = word_tokenize(example_sent)

filtered_sentence = [w for w in word_tokens if not w in stop_words]

filtered_sentence = []

for w in word_tokens:
    if w not in stop_words:
        filtered_sentence.append(w)



In [7]:
import os
import re
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import PorterStemmer
from nltk import WordNetLemmatizer
from num2words import num2words
from collections import Counter
import csv
import tf_idf

In [ ]:
from nltk.book import *


In [ ]:
from nltk.corpus import brown
brown.raw('cp12')


In [7]:

def remove_stopwords(data):
    stop_words = set(stopwords.words('english'))

    word_tokens = word_tokenize(str(data))
    
    filtered_data = ""
    
    for w in word_tokens:
        if w not in stop_words and len(w)>1:
            filtered_data=filtered_data+" "+w
    return filtered_data
    
def remove_punctuation(data):
    marks="~!@#$%^&*()_+=-`[]\;'./{}|:<>?""'\n"
    
    for i in marks:
        data=np.char.replace(data,i,' ')
        data=np.char.replace(data,"  "," ")
    
    data=np.char.replace(data,",",'')
    return data

def remove_apostrophe(data):
    return np.char.replace(data, "'", "")


def stemming(data):
    stemmer= PorterStemmer()
    
    tokens = word_tokenize(str(data))
    
    new_text = ""
    
    for w in tokens:
        new_text = new_text + " " + stemmer.stem(w)
    
    return new_text

def lemmatize(data):
    wnl=WordNetLemmatizer()
    tokens = word_tokenize(str(data))
    
    new_text = ""
    
    for w in tokens:
        new_text = new_text + " " + wnl.lemmatize(w)
    
    return new_text


def convert_numbers(data):
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        
        try:
            w = num2words(int(w))
        except:
            a = 0
        new_text = new_text + " " + w
    new_text = np.char.replace(new_text, "-", " ")
    return new_text

def preprocess(data):
    data=np.char.lower(data)
    
    data = remove_punctuation(data) #remove comma seperately
    
    data = remove_apostrophe(data)
    
    data = remove_stopwords(data)
    
    data = convert_numbers(data)
    
    data = stemming(data)
    
    data = remove_punctuation(data)
    data = convert_numbers(data)
    data = stemming(data) #needed again as we need to stem the words
    data = remove_punctuation(data) #needed again as num2word is giving few hypens and commas fourty-one
    data = remove_stopwords(data) #needed again as num2word is giving stop words 101 - one hundred and one
    return data



In [ ]:
# print(remove_punctuation("hey! how are you? i have 3 colors: red, white, and yellow."))

# print(lemmatize("hey! how are you lying i? have 3 studies colors: red, white, and yellow."))
# print(PorterStemmer().stem("study"))

# print(convert_numbers("hii 1002"))
# file = open('D:\\SDP/stories/SRE/sre01.txt', 'r', encoding="utf8", errors='ignore')
# text = file.read().strip()
# file.close()
#print(preprocess(text))

In [10]:

#extracting all folders
folders=[x[0] for x in os.walk(str(os.getcwd())+'/stories/')]
folders=folders[1:]
#print(folders)

#extracting name and title of each story
dataset=[]
for f in folders:
    file=open(f+'/index.html','r')
    text=file.read().strip()
    file.close()
    
    names=re.findall('><A HREF="(.*)">',text)
    titles=re.findall('<BR><TD> (.*)\n', text)
    for j in range(len(names)):
        dataset.append((str(f)+'/'+names[j],titles[j]))
#print(dataset)

N=len(dataset)
#extracting data
processed_text = []
processed_title = []

for i in dataset[:N]:
    file = open(i[0], 'r', encoding="utf8", errors='ignore')
    text = file.read().strip()
    file.close()
    #preprocess and append whole file i.e. story
    processed_text.append(word_tokenize(str(preprocess(text))))
    
    #preprocess and append title i.e. in i[1]..dataset(filename,title)
    processed_title.append(word_tokenize(str(preprocess(i[1]))))


#calculate df for all words    
DF = {}

for i in range(N):
    tokens = processed_text[i]
    for w in tokens:
        try:
            DF[w].add(i)
        except:
            DF[w] = {i}
    
    tokens = processed_title[i]
    for w in tokens:
        try:
            DF[w].add(i)
        except:
            DF[w] = {i}
for i in DF:
    DF[i] = len(DF[i])
# print("DF",DF)


doc = 0

tf_idf = {}

for i in range(N):
    
    tokens = processed_text[i]
    
    counter = Counter(tokens + processed_title[i])
    words_count = len(tokens + processed_title[i])
    
    for token in np.unique(tokens):
        
        tf = counter[token]/words_count
        df = 0
        try:
            df=DF[token]
        except:
            pass
        idf = np.log((N+1)/(df+1))
        
        tf_idf[doc, token] = tf*idf

    doc += 1



0.0


In [11]:
print(tf_idf[0,"alway"])

0.00025662985945743163


In [8]:
import json
from tqdm import tqdm
import pandas as pd
dataset1=[]
dataset=[]
#dataset1: the whole dataset
#dataset: minimized version containing 3 books
#0:wikipediaId 1:fiebaseId 2:name 3:author 4:publish date 5:genres 6:summary
with open('D:\\SDP/booksummaries.txt', 'r', encoding="utf8", errors='ignore') as f:
    reader=csv.reader(f,dialect='excel-tab')
    for row in reader:
        dataset1.append([row[0],row[2],row[3],row[4],row[5],row[6]])
        



In [9]:
with open('D:\\SDP/booksummaries1.txt', 'r', encoding="utf8", errors='ignore') as f:
    reader=csv.reader(f,dialect='excel-tab')
    for row in reader:
        dataset.append([row[0],row[2],row[3],row[4],row[5],row[6]])
        


In [10]:
df=pd.DataFrame(dataset,columns=['ID','Name','Author','Date','Genres','Summary'])


In [16]:
print(len(dataset))
print(len(df))
df

4
3


,ID,Name,Author,Date,Genres,Summary
0,620,Animal Farm,George Orwell,1945-08-17,"{""/m/016lj8"": ""Roman \u00e0 clef"", ""/m/06nbt"":...","Old Major, the old boar on the Manor Farm, ca..."
1,843,A Clockwork Orange,Anthony Burgess,1962,"{""/m/06n90"": ""Science Fiction"", ""/m/0l67h"": ""N...","Alex, a teenager living in near-future Englan..."
2,986,The Plague,Albert Camus,1947,"{""/m/02m4t"": ""Existentialism"", ""/m/02xlf"": ""Fi...",The text of The Plague is divided into five p...


In [12]:
df['Genres']

0    {"/m/016lj8": "Roman \u00e0 clef", "/m/06nbt":...
1    {"/m/06n90": "Science Fiction", "/m/0l67h": "N...
2    {"/m/02m4t": "Existentialism", "/m/02xlf": "Fi...
3                                                     
Name: Genres, dtype: object

In [13]:
df=df.drop(df[df['Genres']==''].index)


In [14]:
genres={}

print(len(df['Genres']))
for i in df['Genres']:
    for i1,j1 in json.loads(i).items():
        try:
            genres[i1]=j1
        except:
            pass

genres

3


{'/m/016lj8': 'Roman à clef',
 '/m/06nbt': 'Satire',
 '/m/0dwly': "Children's literature",
 '/m/014dfn': 'Speculative fiction',
 '/m/02xlf': 'Fiction',
 '/m/06n90': 'Science Fiction',
 '/m/0l67h': 'Novella',
 '/m/0c082': 'Utopian and dystopian fiction',
 '/m/02m4t': 'Existentialism',
 '/m/0pym5': 'Absurdist fiction',
 '/m/05hgj': 'Novel'}

In [15]:
len(genres)

11

In [29]:
# for x in df:
#     features=tf_idf.generate_features(x[4])
#     print(df[x])

df['Summary'].apply(lambda x: tf_idf.generate_features(x))


['uniting',
 'workhorse',
 'special',
 'thin',
 'working',
 'worlds',
 'tell',
 'upright',
 'violent',
 'work']

['thank']